## 2.2.1 导入工具包

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import warnings

warnings.filterwarnings("ignore")
%matplotlib inline

## 2.2.2 导入数据集

In [2]:
train_data_file = "./zhengqi_train.txt"
test_data_file = "./zhengqi_test.txt"

train_data = pd.read_csv(train_data_file, sep="\t",encoding="utf-8")
test_data = pd.read_csv(test_data_file, sep="\t", encoding="utf-8")

# 4 模型训练


## 4.2.1 导入相关库

In [3]:
from sklearn.linear_model import LinearRegression  #线性回归
from sklearn.neighbors import KNeighborsRegressor  #K近邻回归
from sklearn.tree import DecisionTreeRegressor     #决策树回归
from sklearn.ensemble import RandomForestRegressor #随机森林回归
from sklearn.svm import SVR  #支持向量回归
import lightgbm as lgb #lightGbm模型

from sklearn.model_selection import train_test_split # 切分数据
from sklearn.metrics import mean_squared_error #评价指标

from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

## 4.2.2 切分数据

In [4]:
from sklearn import preprocessing

features_columns = [col for col in train_data.columns if col not in ['target']]
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = min_max_scaler.fit(train_data[features_columns]) 

train_data_scaler = min_max_scaler.transform(train_data[features_columns])
test_data_scaler = min_max_scaler.transform(test_data[features_columns])

train_data_scaler = pd.DataFrame(train_data_scaler)
train_data_scaler.columns = features_columns

test_data_scaler = pd.DataFrame(test_data_scaler)
test_data_scaler.columns = features_columns
train_data_scaler['target'] = train_data['target']

#display(train_data_scaler.describe())
#display(test_data_scaler.describe())

from sklearn.decomposition import PCA

pca = PCA(n_components = 0.9)
new_train_pca_90 = pca.fit_transform(train_data_scaler.iloc[:,0:-1])
new_test_pca_90 = pca.transform(test_data_scaler)
new_train_pca_90 = pd.DataFrame(new_train_pca_90)
new_test_pca_90 = pd.DataFrame(new_test_pca_90)
new_train_pca_90['target'] = train_data_scaler['target']
#display(new_train_pca_90.describe())

#保留16个主成分
pca = PCA(n_components=0.95)
new_train_pca_16 = pca.fit_transform(train_data_scaler.iloc[:,0:-1])
new_test_pca_16 = pca.transform(test_data_scaler)
new_train_pca_16 = pd.DataFrame(new_train_pca_16)
new_test_pca_16 = pd.DataFrame(new_test_pca_16)
new_train_pca_16['target'] = train_data_scaler['target']
new_train_pca_16.describe()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,target
count,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,...,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2.888000e+03,2888.000000
mean,1.039555e-16,6.727459e-18,3.836574e-17,3.190738e-18,6.004738e-17,2.867820e-17,-1.232086e-17,2.838027e-17,-3.419472e-17,1.345011e-17,...,-5.381968e-19,-1.659760e-17,2.848599e-17,-3.500922e-17,-3.435569e-17,1.395467e-17,-8.373765e-17,4.433343e-17,-6.958115e-18,0.126353
std,3.985421e-01,3.490520e-01,2.938955e-01,2.722600e-01,2.035064e-01,1.961281e-01,1.861363e-01,1.607591e-01,1.513411e-01,1.442443e-01,...,1.289251e-01,1.181146e-01,1.148697e-01,1.113406e-01,1.018260e-01,9.585053e-02,9.149580e-02,8.393826e-02,8.190898e-02,0.983966
min,-1.070204e+00,-9.382404e-01,-1.019585e+00,-6.853347e-01,-7.648509e-01,-5.531376e-01,-5.727510e-01,-5.914083e-01,-6.297522e-01,-4.842131e-01,...,-5.773973e-01,-4.258712e-01,-4.576412e-01,-3.888960e-01,-3.658579e-01,-4.574524e-01,-3.528615e-01,-3.401657e-01,-2.816284e-01,-3.044000
25%,-2.802786e-01,-2.603790e-01,-2.062572e-01,-1.971748e-01,-1.250776e-01,-1.246339e-01,-1.256336e-01,-1.009689e-01,-9.717828e-02,-9.245411e-02,...,-7.805577e-02,-7.327538e-02,-7.525674e-02,-7.156570e-02,-6.565231e-02,-5.748351e-02,-6.059756e-02,-5.280777e-02,-5.070524e-02,-0.350250
50%,-1.551054e-02,-8.985867e-03,-1.862128e-02,-2.277698e-02,-4.541365e-03,-1.473143e-02,1.033570e-04,-5.335135e-03,2.214191e-03,-2.135596e-03,...,-5.016301e-03,-7.593646e-03,6.604729e-05,-2.201764e-03,-1.059611e-03,-4.374851e-03,2.924076e-03,-1.280537e-03,7.265520e-04,0.313000
75%,2.275077e-01,2.322535e-01,2.133431e-01,1.680986e-01,1.236219e-01,1.020631e-01,1.271116e-01,9.692888e-02,1.005325e-01,9.123200e-02,...,7.162226e-02,6.871951e-02,7.942442e-02,6.896065e-02,6.298074e-02,5.666983e-02,6.048361e-02,5.009373e-02,5.425216e-02,0.793250
max,1.603357e+00,1.351734e+00,1.021067e+00,1.423139e+00,9.969989e-01,1.301563e+00,6.259154e-01,7.454770e-01,6.420087e-01,5.824542e-01,...,6.543877e-01,5.559756e-01,5.139705e-01,5.040193e-01,4.505913e-01,5.214863e-01,3.540508e-01,4.409993e-01,3.503572e-01,2.538000


In [5]:
#采用 pca 保留16维特征的数据
new_train_pca_16 = new_train_pca_16.fillna(0)
train = new_train_pca_16[new_test_pca_16.columns]
target = new_train_pca_16['target']

# 切分数据 训练数据80% 验证数据20%
train_data,test_data,train_target,test_target=train_test_split(train,target,test_size=0.2,random_state=0)

## 4.2.3 多元线性回归

In [6]:
clf = LinearRegression()
clf.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("LinearRegression:   ", score)

LinearRegression:    0.1346823608106283


## 4.2.4 K近邻算法

In [7]:
clf = KNeighborsRegressor(n_neighbors=8)
clf.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("KNeighborsRegressor: ", score)

KNeighborsRegressor:  0.20105180414684257


## 4.2.5 随机森林回归

In [8]:
clf = RandomForestRegressor(n_estimators=200) # 200棵树模型
clf.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("RandomForestRegressor:   ", score)

RandomForestRegressor:    0.15927152624519897


## 4.2.x 决策树

In [9]:
clf = DecisionTreeRegressor() 
clf.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("DecisionTreeRegressor:   ", score)

DecisionTreeRegressor:    0.338023223183391


## 4.2.6 LGB 模型回归

In [10]:
clf = lgb.LGBMRegressor(
    learning_rate = 0.01,
    max_depth=-1,
    n_estimators=5000,
    boosting_type='gbdt',
    random_state=2019,
    objective='regression'
)
clf.fit(X=train_data, y=train_target, eval_metric="MSE", verbose=50)

score = mean_squared_error(test_target, clf.predict(test_data))
print("lightGbm:  ", score)

lightGbm:   0.13718190135507638
